In [19]:
import xmlrpc.client

ODOO_URL = "https://www.donssonusa.com"
ODOO_DB = "donsson-filters-florida-llc"
ODOO_USERNAME = "gbetancourt@donsson.com"
ODOO_PASSWORD = "DonssonFloridaCol"

# Conexión
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# Ver campos disponibles de un modelo (por ejemplo, productos)
fields = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    'account.move.line', 'fields_get',
    [], {'attributes': ['string', 'type', 'required']}
)

for field, info in sorted(fields.items(), key=lambda x: x[1].get("string", "")):
    print(f"{field:40} | {info.get('type', ''):10} | {info.get('string', '')}")


account_id                               | many2one   | Account
is_account_reconcile                     | boolean    | Account Reconcile
account_root_id                          | many2one   | Account Root
amount_currency                          | monetary   | Amount in Currency
analytic_distribution                    | json       | Analytic Distribution
analytic_distribution_search             | json       | Analytic Distribution Search
analytic_precision                       | integer    | Analytic Precision
analytic_line_ids                        | one2many   | Analytic lines
balance                                  | monetary   | Balance
tax_base_amount                          | monetary   | Base Amount
product_uom_category_id                  | many2one   | Category
cogs_origin_id                           | many2one   | Cogs Origin
company_id                               | many2one   | Company
company_currency_id                      | many2one   | Company Currency
is_stor

In [20]:
import xmlrpc.client
import pandas as pd

ODOO_URL = "https://www.donssonusa.com"
ODOO_DB = "donsson-filters-florida-llc"
ODOO_USERNAME = "gbetancourt@donsson.com"
ODOO_PASSWORD = "DonssonFloridaCol"

# Conexión
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

quant_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    'stock.quant', 'search',
    [[['quantity', '>', 0]]]   # o sin filtro: []
)

quants = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    'stock.quant', 'read',
    [quant_ids],
    {'fields': [
        'product_id',
        'location_id',
        'quantity',
        'reserved_quantity',
        'available_quantity'
    ]}
)

# --- Convertir a DataFrame ---
df = pd.DataFrame(quants)

# Expandir campos many2one
df['product_id'] = df['product_id'].apply(lambda x: x[1] if x else None)
df['location_id'] = df['location_id'].apply(lambda x: x[1] if x else None)

df = df[df['location_id'].isin(['WH/Stock', 'WH/Amazon'])]

# --- 3) Agrupar por producto y sumar quantity ---
df_final = (
    df.groupby('product_id', as_index=False)
      .agg(quantity_total=('quantity', 'sum'))
)


In [21]:
df[df["product_id"].str.contains("2671")]

,id,product_id,location_id,quantity,reserved_quantity,available_quantity
59,244,[DA2671] DA2671 AIR FILTER,WH/Stock,94.0,0.0,94.0


In [22]:
df_inv_actual = df_final.copy()

In [23]:
import xmlrpc.client
import pandas as pd

ODOO_URL = "https://www.donssonusa.com"
ODOO_DB = "donsson-filters-florida-llc"
ODOO_USERNAME = "gbetancourt@donsson.com"
ODOO_PASSWORD = "DonssonFloridaCol"

# Conexión
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")
#  --- 1) Leer sale.report (ventas confirmadas y entregadas) ---
report_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    'sale.report', 'search',
    [[
        ['state', 'in', ['sale', 'done']],
        ['product_uom_qty', '>', 0],
        ['date', '>=', '2024-01-01']
    ]]
)

data = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    'sale.report', 'read',
    [report_ids],
    {'fields': [
        'product_id',
        'product_uom_qty',
        'date',     # fecha del pedido
        'partner_id'
    ]}
)

df = pd.DataFrame(data)

# Expandir campos many2one
df['product'] = df['product_id'].apply(lambda x: x[1] if isinstance(x, list) else None)
df['cliente'] = df['partner_id'].apply(lambda x: x[1] if isinstance(x, list) else None)

# Convertir fecha a mes (AAAA-MM)
df['date'] = pd.to_datetime(df['date'])
df['mes'] = df['date'].dt.to_period('M').astype(str)

# --- 2) Agrupar unidades vendidas por producto y mes ---
df_mes = (
    df.groupby(['product', 'mes'], as_index=False)
      .agg(unidades_vendidas=('product_uom_qty', 'sum'))
      .sort_values(['product', 'mes'])
)


In [24]:
df_mes[df_mes["product"].str.contains("2671")]

,product,mes,unidades_vendidas
353,DA2671KIT,2024-02,1.0
354,DA2671KIT,2024-03,1.0
355,DA2671KIT,2024-06,60.0
749,[DA2671] DA2671 AIR FILTER,2024-01,3.0
750,[DA2671] DA2671 AIR FILTER,2024-04,10.0
751,[DA2671] DA2671 AIR FILTER,2024-05,13.0
752,[DA2671] DA2671 AIR FILTER,2024-06,81.0
753,[DA2671] DA2671 AIR FILTER,2024-07,2.0
754,[DA2671] DA2671 AIR FILTER,2024-08,4.0
755,[DA2671] DA2671 AIR FILTER,2024-09,6.0


In [25]:
df_inv_actual.head()

,product_id,quantity_total
0,BALDWIN AIR FILTER PA2807,2.0
1,BALDWIN B1400,8.0
2,BALDWIN B495,2.0
3,BALDWIN B7152,2.0
4,BALDWIN B7299,1.0


In [26]:
import pandas as pd

# -----------------------------------------------------
# 1. Normalizar columnas
# -----------------------------------------------------
# df_mes tiene product_id? se renombra
if "product_id" in df_mes.columns:
    df_mes = df_mes.rename(columns={"product_id": "product"})

# df_inv_actual tiene product_id (es el nombre del producto)
df_inv_actual = df_inv_actual.rename(columns={
    "product_id": "product",
    "quantity_total": "inventario_actual"
})

# -----------------------------------------------------
# 2. Filtrar desde enero 2024 hasta octubre 2025
# -----------------------------------------------------
df_mes["mes"] = pd.to_datetime(df_mes["mes"])
df_filtrado = df_mes[
    (df_mes["mes"] >= "2024-01-01") &
    (df_mes["mes"] <= "2025-10-31")
].copy()

# Crear columna formato AAAA-MM
df_filtrado["mes_str"] = df_filtrado["mes"].dt.to_period("M").astype(str)

# -----------------------------------------------------
# 3. Pivot ventas mensuales por producto
# -----------------------------------------------------
pivot = df_filtrado.pivot_table(
    index="product",
    columns="mes_str",
    values="unidades_vendidas",
    aggfunc="sum",
    fill_value=0
).reset_index()  # <-- AQUÍ se arregla para que product sea columna

# -----------------------------------------------------
# 4. Unir inventario actual
# -----------------------------------------------------
pivot = pivot.merge(df_inv_actual, on="product", how="left")

# -----------------------------------------------------
# 5. Calcular promedio mensual
# -----------------------------------------------------
columnas_meses = [
    c for c in pivot.columns
    if "-" in c  # solo columnas tipo AAAA-MM
]

pivot["promedio_mensual"] = pivot[columnas_meses].mean(axis=1).round(2)

# -----------------------------------------------------
# 6. Exportar Excel visible con todas las columnas
# -----------------------------------------------------
ruta_excel = "/home/donsson/proyectos/DONSSON MIAMI/listas_precios/inventario_y_ventas.xlsx"

with pd.ExcelWriter(ruta_excel, engine="openpyxl") as writer:
    pivot.to_excel(writer, sheet_name="Pivot", index=False)
    df_filtrado.to_excel(writer, sheet_name="Ventas_Detalle", index=False)
    df_inv_actual.to_excel(writer, sheet_name="Inventario", index=False)

print("Excel generado en:", ruta_excel)





Excel generado en: /home/donsson/proyectos/DONSSON MIAMI/listas_precios/inventario_y_ventas.xlsx


In [28]:
pivot[pivot["product"].str.contains("2671")]

,product,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,...,2025-03,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,inventario_actual,promedio_mensual
201,DA2671KIT,0.0,1.0,1.0,0.0,0.0,60.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2.82
286,[DA2671] DA2671 AIR FILTER,3.0,0.0,0.0,10.0,13.0,81.0,2.0,4.0,6.0,...,65.0,18.0,1.0,5.0,2.0,0.0,102.0,0.0,94.0,14.77
